In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

constructor_results = pd.read_csv('constructor_results.csv')
df = pd.DataFrame(constructor_results)
constructor_standings = pd.read_csv('constructor_standings.csv')
constructors = pd.read_csv('constructors.csv')
races = pd.read_csv('races.csv')
results = pd.read_csv('results.csv')

#merging of data frames to get one dataset with desired columns for the model
df = pd.merge(constructor_results, results, on=['raceId', 'constructorId'], suffixes=('_constructor', '_results'))
df = pd.merge(df, races, on='raceId')
df = pd.merge(df, constructors, on='constructorId')
df = pd.merge(df, constructor_standings, on=['raceId', 'constructorId'], suffixes=('_constructor', '_standings'))

#first 5 rows of the merged dataframe
print(df.head())

#descriptive statistics for each column of the dataframe
print(df.describe())

#changing the model's dataset to only include data between the years 1961 and 1990
df = df[(df['year'] >= 1961) & (df['year'] <= 1990)]

#desired features for the model's dataset
selected_columns = [
    'grid', 'laps',  
    'position_constructor',   
    'year', 'round', 'circuitId', 'points', 'position_standings', 'wins'  
]

#model only has desired features
df = df[selected_columns]

#converting categorical columns into numerical columns
df = pd.get_dummies(df)

#dropping all rows where at least 1 element is missing
#df = df.dropna()

#dataset of the predictor variables
X = df.drop('points', axis=1)

#series representing the target variable
y = df['points']

#evening the distribution of values of the predictor variable
y= np.log1p(y)

#machine learning model 
model = RandomForestRegressor(n_estimators = 100, max_depth = 20, min_samples_leaf = 1, min_samples_split = 2, random_state = 42)


#training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

#fitting the training data to the model 
model.fit(X_train, y_train)



#predicted values
y_pred = model.predict(X_test)

#efficiency of model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

#importance of each feature in dataset when making predictions
importance_of_features = model.feature_importances_

#predictor variables
X_columns = X.columns

#list paring predictor variable name with its importance to making predictions
importance_pairs = list(zip(importance_of_features, X_columns))

#sorted list of predictor variables in order of importance to making predicitions
sorted_important_pairs = sorted(importance_pairs, key = lambda x:x[0], reverse = True)

"""
loops through all variables in sorted_imported_pairs in order to print
the variable name with its corresponding importance to making predicitons
"""

for importance, variable in sorted_important_pairs:
    print(f"{variable}: {importance}")


"""
scatter plot to compare all actual points to their corresponding
predicted points between the years of 1961 and 1990
"""

plt.figure(figsize=(12, 8))
plt.scatter(X_test['year'], np.expm1(y_test), label='Actual Points', alpha=0.7)
plt.scatter(X_test['year'], np.expm1(y_pred), label='Predicted Points', alpha=0.7)
plt.title('Actual vs Predicted Points Over Time')
plt.xlabel('Year')
plt.ylabel('Points')
plt.legend()
plt.grid(True)
plt.show()

   constructorResultsId  raceId  constructorId  points_constructor status  \
0                     1      18              1                14.0     \N   
1                     1      18              1                14.0     \N   
2                    14      19              1                10.0     \N   
3                    14      19              1                10.0     \N   
4                    25      20              1                 4.0     \N   

   resultId  driverId number  grid position_constructor ...  sprint_time  \
0         1         1     22     1                    1 ...           \N   
1         5         5     23     3                    5 ...           \N   
2        25         5     23     8                    3 ...           \N   
3        27         1     22     9                    5 ...           \N   
4        49         5     23     5                    5 ...           \N   

   constructorRef   name_y  nationality                                 url_y  \

<Figure size 1200x800 with 1 Axes>